In [1]:
import pickle
import numpy as np
from scipy import optimize

# Results and stats for Uninformed

In [43]:
""" Stats of Uninformed """
method = "Uninformed"
name_data = "mnist" # mnist or fmnist
model = "fcn" # fcn or cnn
objective = "fclassic"
seeds = np.arange(5)
n_bound = 60000
mc_samples = 60000
delta_test=0.01
delta=0.025

kln = []
risks = []
train_loss = []
eval_loss = []
test_loss = []
for seed in seeds:
    exp_settings = f"{name_data}_{model}_{objective}_{seed}.pt"
    results_dir = f"./results/{method}/results_" + exp_settings

    with open(results_dir, "rb") as handle:
        result_seed = pickle.load(handle)
    ## read results
    risks.append(result_seed["risk"])
    train_loss.append(result_seed["train_loss"])
    eval_loss.append(result_seed["train_loss"]) # should be eval_loss
    test_loss.append(result_seed["test_loss"])
    kln.append(result_seed["kl"]/n_bound)

# basic stats
print("---- basic stats ----")
print("train_loss: ", np.mean(train_loss), " ; ", np.std(train_loss))
print("test_loss: ", np.mean(test_loss), " ; ", np.std(test_loss))
print("risks: ", np.mean(risks), " ; ", np.std(risks))

# further details
print("---- further details ----")
## train loss
print("train_loss: ", np.mean(train_loss), " ; ", np.std(train_loss))
## for bound
print("eval_loss: ", np.mean(eval_loss), " ; ", np.std(eval_loss))
print("kln: ", np.mean(kln)," ; ", np.std(kln))
print("risks: ", np.mean(risks), " ; ", np.std(risks))
## test loss
print("test_loss: ", np.mean(test_loss), " ; ", np.std(test_loss))

---- basic stats ----
train_loss:  0.3655233333333333  ;  0.0019627814730914716
test_loss:  0.35832  ;  0.003497656358191872
risks:  0.4778762421327808  ;  0.001957785750838015
---- further details ----
train_loss:  0.3655233333333333  ;  0.0019627814730914716
eval_loss:  0.3655233333333333  ;  0.0019627814730914716
kln:  0.022833062337239585  ;  7.562504371881661e-05
risks:  0.4778762421327808  ;  0.001957785750838015
test_loss:  0.35832  ;  0.003497656358191872


# Results and stats for Informed

In [44]:
""" Stats of Informed """
method = "Informed"
name_data = "mnist"
model = "fcn"
objective = "fclassic"
seeds = np.arange(5)
n_bound = 30000
mc_samples = 30000
delta_test=0.01
delta=0.025

kln = []
risks = []
train_loss = []
eval_loss = []
test_loss = []
for seed in seeds:
    exp_settings = f"{name_data}_{model}_{objective}_{seed}.pt"
    results_dir = f"./results/{method}/results_" + exp_settings

    with open(results_dir, "rb") as handle:
        result_seed = pickle.load(handle)
    
    print("result_seed: ", seed, result_seed)
    ## read results
    risks.append(result_seed["risk"])
    train_loss.append(result_seed["train_loss"])
    eval_loss.append(result_seed["eval_loss"])
    test_loss.append(result_seed["test_loss"])
    kln.append(result_seed["kl"]/n_bound)

# basic stats
print("---- basic stats ----")
print("train_loss: ", np.mean(train_loss), " ; ", np.std(train_loss))
print("test_loss: ", np.mean(test_loss), " ; ", np.std(test_loss))
print("risks: ", np.mean(risks), " ; ", np.std(risks))

# further details
print("---- further details ----")
## Train loss
print("train_loss: ", np.mean(train_loss), " ; ", np.std(train_loss))
## For bound
print("eval_loss: ", np.mean(eval_loss), " ; ", np.std(eval_loss))
print("kln: ", np.mean(kln)," ; ", np.std(kln))
print("risks: ", np.mean(risks), " ; ", np.std(risks))
## Test loss
print("test_loss: ", np.mean(test_loss), " ; ", np.std(test_loss))

result_seed:  0 {'kl': array(80.83292, dtype=float32), 'risk': 0.3837803676549357, 'train_loss': 0.3415, 'eval_loss': 0.3379666666666667, 'test_loss': 0.34}
result_seed:  1 {'kl': array(86.555855, dtype=float32), 'risk': 0.38910774574478296, 'train_loss': 0.34203333333333336, 'eval_loss': 0.342, 'test_loss': 0.3291}
result_seed:  2 {'kl': array(81.00739, dtype=float32), 'risk': 0.3796732592069954, 'train_loss': 0.3402, 'eval_loss': 0.33393333333333336, 'test_loss': 0.3377}
result_seed:  3 {'kl': array(82.6727, dtype=float32), 'risk': 0.3849526530601304, 'train_loss': 0.33741666666666664, 'eval_loss': 0.33873333333333333, 'test_loss': 0.3328}
result_seed:  4 {'kl': array(80.542496, dtype=float32), 'risk': 0.38673049442256374, 'train_loss': 0.3463833333333333, 'eval_loss': 0.3409, 'test_loss': 0.3394}
---- basic stats ----
train_loss:  0.3415066666666667  ;  0.002915296821175431
test_loss:  0.3358  ;  0.004197618372363074
risks:  0.38484890401788163  ;  0.003150172958329321
---- further 

# Results and stats for InformedExcess

In [45]:
""" Stats of InformedExcess """
method = "InformedExcess"
name_data = "mnist" # mnist or fmnist
model = "fcn" # fcn or cnn
objective = "fclassic"
seeds = np.arange(5)
n_bound = 30000
mc_samples = 30000
delta_test=0.01
delta=0.025
# Excess loss
rv = np.array([-1, 0, 1])
js = rv[1:]
js_minus = rv[1:] - rv[0:-1]

kln = [] # kl/n
train_loss = []
eval_loss = []
excess_loss = []
h_loss = []
test_loss = []
risks = [] # the risk
excess_risks = [] # the excess risk
h_risks = [] # reference classifier h risks

for seed in seeds:
    exp_settings = f"{name_data}_{model}_{objective}_{seed}.pt"
    results_dir = f"./results/{method}/results_" + exp_settings

    with open(results_dir, "rb") as handle:
        result_seed = pickle.load(handle)
    print("result_seed: ", seed, result_seed)

    # Basic
    risks.append(result_seed["risk"][0])
    h_risks.append(result_seed["risk"][1])
    excess_risks.append(result_seed["risk"][0] - result_seed["risk"][1])
    train_loss.append(result_seed["train_loss"])
    test_loss.append(result_seed["test_loss"])

    # excess risk
    excess_loss_1_seed = result_seed["risk"][2][0]
    excess_loss_2_seed = result_seed["risk"][2][1]
    excess_loss_seed = rv[0] + js_minus[0] * excess_loss_1_seed + js_minus[1] * excess_loss_2_seed
    excess_loss.append(excess_loss_seed)
    kl_seed = result_seed["kl"] ; kln.append(kl_seed/n_bound)

    # h risk
    h_loss_seed = result_seed["risk"][3] ; h_loss.append(h_loss_seed)

# basic stats
print("---- basic stats ----")
print("train_loss: ", np.mean(train_loss), " ; ", np.std(train_loss))
print("test_loss: ", np.mean(test_loss), " ; ", np.std(test_loss))
print("risks: ", np.mean(risks), " ; ", np.std(risks))

# further details
print("---- further details ----")
## Train loss
print("train_loss: ", np.mean(train_loss), " ; ", np.std(train_loss))
## For bound
### excess bound
print("kln: ", np.mean(kln)," ; ", np.std(kln))
print("excess_loss: ", np.mean(excess_loss), " ; ", np.std(excess_loss))
print("excess_risk: ", np.mean(excess_risks)," ; ", np.std(excess_risks))
### h bound
print("h_loss: ", np.mean(h_loss), " ; ", np.std(h_loss))
print("h_risks: ", np.mean(h_risks), " ; ", np.std(h_risks))
### bound
print("risks: ", np.mean(risks), " ; ", np.std(risks))
## Test loss
print("test_loss: ", np.mean(test_loss), " ; ", np.std(test_loss))

result_seed:  0 {'kl': array(2257.233, dtype=float32), 'risk': (0.3705466478844455, 0.027586265624620698, array([0.99473333, 0.162     ], dtype=float32), 0.024833333333333332), 'train_loss': 0.17886666666666667, 'eval_loss': 0.1956, 'test_loss': 0.1728}
result_seed:  1 {'kl': array(2294.978, dtype=float32), 'risk': (0.3520138451579917, 0.02772651851628119, array([0.995     , 0.14586666], dtype=float32), 0.024966666666666668), 'train_loss': 0.16718333333333332, 'eval_loss': 0.18523333333333333, 'test_loss': 0.1628}
result_seed:  2 {'kl': array(2210.3599, dtype=float32), 'risk': (0.35870430111039253, 0.028182203222536065, array([0.995     , 0.15366666], dtype=float32), 0.0254), 'train_loss': 0.17398333333333332, 'eval_loss': 0.1913, 'test_loss': 0.1704}
result_seed:  3 {'kl': array(2274.2197, dtype=float32), 'risk': (0.35437947337471476, 0.02737584847988072, array([0.9948, 0.1487], dtype=float32), 0.024633333333333333), 'train_loss': 0.16993333333333333, 'eval_loss': 0.18553333333333333,

# Results and stats for Recursive PAC-Bayes

In [54]:
method = "rpb"
name_data = "mnist" # mnist or fmnist
model = "fcn" # fcn or cnn
layers = 4
objective = "fclassic"

T = 8 # 2, 4, 6, or 8
if T == 2:
    split = "uniform"
else:
    split = "geometric"

n_train = 60000
if split == "uniform":
    T_splits = [int(n_train / T)] * (T-1)
    T_splits.append(n_train - int(n_train / T)*(T-1))
elif split == "geometric":
    if T == 2:
        T_splits = [20000, 40000]
    elif T == 4:
        T_splits = [7500, 7500, 15000, 30000]
    elif T == 6:
        T_splits = [1875, 1875, 3750, 7500, 15000, 30000]
    elif T == 8:
        T_splits = [468, 469, 938, 1875, 3750, 7500, 15000, 30000]
n_train_t_cumsum = np.cumsum(T_splits)
n_posteriors = [n_train - n_train_t_cumsum[t - 2] for t in range(1, T + 1)]
n_posteriors[0] = n_train
print("n_posteriors: ", n_posteriors)

gamma_t = 0.5
# Excess loss
rv = np.array([-gamma_t, 0, 1-gamma_t, 1])
js = rv[1:]
js_minus = rv[1:] - rv[0:-1]

recursive_step_1 = False # Use B_1
risk_laststep = False
seeds = np.arange(5)

n_posteriors:  [60000, 59532, 59063, 58125, 56250, 52500, 45000, 30000]


In [55]:
kl_seeds = []
excess_risk_seeds = []
risk_seeds = []
train_loss_seeds = []
test_loss_seeds = []
for seed in seeds:

    exp_settings = f"{name_data}_{model}_{layers}_{objective}_{split}_{T}_{recursive_step_1}_{risk_laststep}_{gamma_t}_{seed}.pt"

    results_dir = f"./results/rpb/results_" + exp_settings
    with open(results_dir, "rb") as handle:
        results = pickle.load(handle)

    kl_seeds.append(results["kl"])
    excess_risk_seeds.append(results["excess_risk"]) # E_t
    risk_seeds.append(results["risk"]) # B_t
    train_loss_seeds.append(results["train_loss"])
    test_loss_seeds.append(results["test_loss"])

kl_seeds = np.array(kl_seeds)
excess_risk_seeds = np.array(excess_risk_seeds)
risk_seeds = np.array(risk_seeds)
train_loss_seeds = np.array(train_loss_seeds)
test_loss_seeds = np.array(test_loss_seeds)

# basic stats
print("---- basic stats ----")
print("train_loss: ", train_loss_seeds.mean(0), " ; ", train_loss_seeds.std(0))
print("test_loss: ", test_loss_seeds.mean(0)[-1], " ; ", test_loss_seeds.std(0)[-1])
print("risks: ", risk_seeds.mean(0)[-1], " ; ", risk_seeds.std(0)[-1])

---- basic stats ----
train_loss:  0.09696666666666667  ;  0.0
test_loss:  0.0942  ;  0.0
risks:  0.1763910859890787  ;  0.0


## Further Details

In [32]:
print("---- further details ----")
print("test_loss in rpb")
print("mean: ", test_loss_seeds.mean(0))
print("std: ", test_loss_seeds.std(0))
print("B_t in rpb")
print("mean: ", risk_seeds.mean(0))
print("std: ", risk_seeds.std(0))
print("E_t in rpb")
print("mean: ", excess_risk_seeds.mean(0))
print("std: ", excess_risk_seeds.std(0))
print("kln in rpb")
print("mean: ", (kl_seeds / n_posteriors).mean(0))
print("std: ", (kl_seeds / n_posteriors).std(0))

---- further details ----
test_loss in rpb
mean:  [0.34166 0.13638 0.11482 0.10584 0.098   0.09554]
std:  [0.00452    0.00320337 0.00216185 0.00217954 0.00186869 0.0017625 ]
B_t in rpb
mean:  [0.45416848 0.38498419 0.30313124 0.24420018 0.20476847 0.18565702]
std:  [0.0057251  0.00335994 0.00181273 0.00193779 0.00162876 0.00202064]
E_t in rpb
mean:  [0.15789995 0.11063915 0.09263456 0.08266838 0.08327278]
std:  [0.00309851 0.0022632  0.00129615 0.00144103 0.0025285 ]
kln in rpb
mean:  [0.01871792 0.05630976 0.00963022 0.00437401 0.00318368 0.00264056]
std:  [0.00024784 0.00110821 0.00034405 0.00028584 0.00010943 0.00012382]


## More on excess losses

In [40]:
results["loss"]

[0.36043333333333333,
 array([0.74940217, 0.14298494, 0.03363441], dtype=float32),
 array([0.9346667 , 0.11953778, 0.04177778], dtype=float32),
 array([0.95125717, 0.11177143, 0.03944762], dtype=float32),
 array([0.9535111 , 0.10024445, 0.0344    ], dtype=float32),
 array([0.9631    , 0.1033    , 0.03763333], dtype=float32)]

In [42]:
print("emp loss for B1 is ", results["loss"][0], " with B_t ", risk_seeds.mean(0)[0])
for t in range(1, T):
    exc = (results["loss"][t] * js_minus).sum(0) + rv[0]
    print("excess loss t = ", t, " is ", exc, " with E_t ", excess_risk_seeds.mean(0)[t-1])

emp loss for B1 is  0.36043333333333333  with B_t  0.4541684779633818
excess loss t =  1  is  -0.0369892418384552  with E_t  0.1578999483098555
excess loss t =  2  is  0.04799112491309643  with E_t  0.11063914958110883
excess loss t =  3  is  0.051238108426332474  with E_t  0.09263455925429699
excess loss t =  4  is  0.044077783823013306  with E_t  0.08266838015835745
excess loss t =  5  is  0.05201667360961437  with E_t  0.08327278171601249


In [65]:

def KL(Q, P):
    """
    Compute Kullback-Leibler (KL) divergence between distributions Q and P.
    """
    return sum([q * np.log(q / p) if q > 0.0 else 0.0 for q, p in zip(Q, P)])


def KL_binomial(q, p):
    """
    Compute the KL-divergence between two Bernoulli distributions of probability
    of success q and p. That is, Q=(q,1-q), P=(p,1-p).
    """
    return KL([q, 1.0 - q], [p, 1.0 - p])


def get_binominal_inv(n, k, delta):
    for p in np.linspace(1, 0, 100001):
        if binom.pmf(k, n, p) >= delta:
            return p


def solve_kl_sup(q, right_hand_side):
    """
    find x such that:
        kl( q || x ) = right_hand_side
        x > q
    """
    f = lambda x: KL_binomial(q, x) - right_hand_side

    if f(1.0 - 1e-9) <= 0.0:
        return 1.0 - 1e-9
    else:
        return optimize.brentq(f, q, 1.0 - 1e-11)